# Automatyczne tworzenie podsumowań

- ekstraktywne
- abstrakcyjne

## Dane

The Polish Summaries Corpus

http://zil.ipipan.waw.pl/PolishSummariesCorpus

In [1]:
import xml.etree.ElementTree as ET
from collections import Counter
import os
import re

In [2]:
os.listdir("PSC_1.0")

['schema.xsd', 'data', 'manual.pdf']

In [3]:
os.listdir("PSC_1.0/data")

['199911100061.xml',
 '199906240088.xml',
 '200101020062.xml',
 '199912210037.xml',
 '199910090047.xml',
 '200103100019.xml',
 '199911130036.xml',
 '200012110071.xml',
 '199704210013.xml',
 '199704250018.xml',
 '200002160046.xml',
 '200107250080.xml',
 '200010190068.xml',
 '199912090047.xml',
 '200002290046.xml',
 '200006260079.xml',
 '199910270011.xml',
 '200001140061.xml',
 '199911250055.xml',
 '199801230001.xml',
 '200110200070.xml',
 '200012160049.xml',
 '199912280024.xml',
 '200010130063.xml',
 '200004220045.xml',
 '200005060030.xml',
 '200106220087.xml',
 '200012080134.xml',
 '200012110070.xml',
 '200004170065.xml',
 '200012120068.xml',
 '200004220048.xml',
 '199801310003.xml',
 '200006130055.xml',
 '200203290078.xml',
 '199912230058.xml',
 '199910020029.xml',
 '200202020036.xml',
 '199911130037.xml',
 '199901300101.xml',
 '200012110069.xml',
 '200012120072.xml',
 '200111150042.xml',
 '200004120022.xml',
 '199901180149.xml',
 '200001180040.xml',
 '200001290045.xml',
 '20010308002

In [4]:
with open("PSC_1.0/data/199911100061.xml", "r") as f:
    example = f.read()

In [5]:
example

'<?xml version=\'1.0\' encoding=\'UTF-8\'?>\n<text id="199911100061">\n  <date>1999-11-10</date>\n  <title>Poszerzanie wolności</title>\n  <section>Publicystyka, Opinie</section>\n  <authors>Jerzy Buzek</authors>\n  <body>ŚWIĘTO NIEPODLEGŁOŚCI\n\nNic nie jest dobitniejszym świadectwem solidarności niż praca dla dobra wspólnego\n\nPoszerzanie wolności\n\nRYS. ANDRZEJ LEGUS\n\nJERZY BUZEK\n\nŚwięto Niepodległości, na przełomie tysiącleci, stwarza szczególną okazję do refleksji nad dokonaniami dwóch pokoleń: tego, którego udziałem stało się w 1918 roku odzyskanie wolności i tego, które niepodległą Rzeczpospolitą od dziesięciu lat współtworzy. Jakie pożytki czerpiemy dziś z ponownie odzyskanej wolności?\n\nJaka jest dzisiejsza wymowa słowa patriotyzm? Jak teraz układają się relacje między Polakami a ich państwem? Na czym w 1999 roku polega czyn obywatelski? Odpowiedzi na te pytania postanowiłem w tym roku poszukać w Krakowie, "mateczniku polskości", w mieście, gdzie historia i współczesnoś

In [6]:
root = ET.fromstring(example)

In [7]:
print(root.find("body").text)

ŚWIĘTO NIEPODLEGŁOŚCI

Nic nie jest dobitniejszym świadectwem solidarności niż praca dla dobra wspólnego

Poszerzanie wolności

RYS. ANDRZEJ LEGUS

JERZY BUZEK

Święto Niepodległości, na przełomie tysiącleci, stwarza szczególną okazję do refleksji nad dokonaniami dwóch pokoleń: tego, którego udziałem stało się w 1918 roku odzyskanie wolności i tego, które niepodległą Rzeczpospolitą od dziesięciu lat współtworzy. Jakie pożytki czerpiemy dziś z ponownie odzyskanej wolności?

Jaka jest dzisiejsza wymowa słowa patriotyzm? Jak teraz układają się relacje między Polakami a ich państwem? Na czym w 1999 roku polega czyn obywatelski? Odpowiedzi na te pytania postanowiłem w tym roku poszukać w Krakowie, "mateczniku polskości", w mieście, gdzie historia i współczesność przenikają się w stopniu najpełniejszym.

Dziedzictwo

11 listopada to wielkie i radosne święto. Odzyskanie wolności zawsze stanowi tytuł do narodowej dumy. Dla Polaków czyn zbrojny, który przyniósł wolność, miał wymiar szczególny. 

In [8]:
for summary in root.find("summaries").findall("summary"):
    print(summary.attrib)
    print(summary.find("body").text)

{'ratio': '10', 'type': 'extract', 'author': 'G'}
11 listopada to wielkie i radosne święto. Dla Polaków czyn zbrojny, który przyniósł wolność, miał wymiar szczególny. Pamięć o tych, którzy mimo upokorzeń i klęsk nigdy nie pogodzili się z utratą wolności, a także o tych, którzy ów protest przechowali przez noc nazizmu i komunizmu na zawsze pozostanie źródłem naszej siły.
Jak dziś postrzegamy czyn patriotyczny? Dziś narodowy czyn ma swój początek w gminie, gdzie podejmowane są decyzje dotyczące lokalnej społeczności. Kultura narodowa i lokalna są ze sobą splecione. 
W małych ojczyznach dobrze zagospodarowano wolność. właśnie tam najsilniejsza jest owa pamięć historyczna. W czasach globalizacji nowoczesny patriotyzm rozumiem właśnie jako budowanie Polski na dole, w lokalnej przestrzeni duchowej. czyn obywatelski jest rozumiany jako aktywność społeczna, a nie jako reakcja na zagrożenia zewnętrzne. Jest to także czyn radosny.
{'ratio': '20', 'type': 'extract', 'author': 'G'}
11 listopada to

In [9]:
PATTERN1 = re.compile(r"(?<=[a-zA-Z])\n")
PATTERN2 = re.compile(r"\s+")

def clean_text(text):
    out = PATTERN1.sub(". ", text)
    return PATTERN2.sub(" ", out)

In [10]:
def load_data(path, n_doc=None):
    data = []
    count = 0
    for file in os.listdir(path):
        with open(f"{path}/{file}", "r") as f:
            raw = f.read()
        root = ET.fromstring(raw)
        doc = clean_text(root.find("body").text)
        for summary in root.find("summaries").findall("summary"):
            if summary.attrib["ratio"] == "10":
                kind = summary.attrib["type"]
                text = clean_text(summary.find("body").text)
                data.append((doc, text, kind))
        if n_doc:
            count += 1
            if count == n_doc:
                break
    return data

In [11]:
data = load_data("PSC_1.0/data", 1)

In [12]:
len(data)

5

In [13]:
data[0]

('ŚWIĘTO NIEPODLEGŁOŚCI. Nic nie jest dobitniejszym świadectwem solidarności niż praca dla dobra wspólnego. Poszerzanie wolności. RYS. ANDRZEJ LEGUS. JERZY BUZEK. Święto Niepodległości, na przełomie tysiącleci, stwarza szczególną okazję do refleksji nad dokonaniami dwóch pokoleń: tego, którego udziałem stało się w 1918 roku odzyskanie wolności i tego, które niepodległą Rzeczpospolitą od dziesięciu lat współtworzy. Jakie pożytki czerpiemy dziś z ponownie odzyskanej wolności? Jaka jest dzisiejsza wymowa słowa patriotyzm? Jak teraz układają się relacje między Polakami a ich państwem? Na czym w 1999 roku polega czyn obywatelski? Odpowiedzi na te pytania postanowiłem w tym roku poszukać w Krakowie, "mateczniku polskości", w mieście, gdzie historia i współczesność przenikają się w stopniu najpełniejszym. Dziedzictwo. 11 listopada to wielkie i radosne święto. Odzyskanie wolności zawsze stanowi tytuł do narodowej dumy. Dla Polaków czyn zbrojny, który przyniósł wolność, miał wymiar szczególny. 